In [1]:
import pandas as pd
import pandas_ta as ta
import config
from oandapyV20 import API
import oandapyV20.endpoints.instruments as instruments
import oandapyV20.endpoints.orders as orders

from datetime import datetime, timezone
import time

In [2]:
# Setup your OANDA connection
client = API(access_token=config.OANDA_API_KEY)

In [3]:
# define variables
timeframe = "M5"
instrument = "GBP_JPY"

In [4]:
def get_candles(tf):
    params = {
        "granularity": tf,
        "price": "A"  # Ask prices
    }

    r = instruments.InstrumentsCandles(instrument=instrument, params=params)
    candles = client.request(r)['candles']

    # Convert to pandas DataFrame
    data = []

    for c in candles:
        if c["complete"]:
            data.append({
                "time": c["time"],
                "open": float(c["ask"]["o"]),
                "high": float(c["ask"]["h"]),
                "low": float(c["ask"]["l"]),
                "close": float(c["ask"]["c"])
            })

    df = pd.DataFrame(data)
    df["time"] = pd.to_datetime(df["time"])
    
    return df

In [5]:
# Function to calculate indicators
def calculate_indicators(df):
    # EMAs
    df["EMA_5"] = ta.ema(df["close"], length=5)
    df["EMA_8"] = ta.ema(df["close"], length=8)

    # ATR
    df["ATR_14"] = ta.atr(df["high"], df["low"], df["close"], length=14)

    return df

In [6]:
# Function to place an order
def place_order(stop_loss, take_profit):
    data = {
        "order": {
            "instrument": instrument,
            "units": 100,
            "type": "MARKET",
            "stopLossOnFill": {"price": f"{stop_loss:.3f}"},
            "takeProfitOnFill": {"price": f"{take_profit:.3f}"},
        }
    }

    r = orders.OrderCreate(config.OANDA_ACCOUNT_ID, data=data)
    client.request(r)
    print(f"Placed order for {instrument} with stop loss at {round(stop_loss, 3)} and take profit at {round(take_profit, 3)}.")

In [7]:
# Function to check for EMA Crossover signals
def ema_crossover(df):
    tp_ratio = 1.5

    # Check if the 5-period EMA crosses above the 8-period EMA
    last_candle = df.iloc[-1]
    previous_candle = df.iloc[-2]

    # Crossover Buy Signal (EMA 5 crosses above EMA 8)
    if last_candle['EMA_5'] > last_candle['EMA_8'] and previous_candle['EMA_5'] < previous_candle['EMA_8']:
        print("Buy Signal: EMA 5 crossed above EMA 8")
        entry_price = last_candle['close']
        stop_loss = entry_price - last_candle['ATR_14']
        stop_distance = entry_price - stop_loss
        take_profit = entry_price + (stop_distance * tp_ratio)
        place_order(stop_loss, take_profit)
    else:
        print("Strategy conditions not met")

In [8]:
def run_bot():
    print("Starting trading bot")
    last_checked = None

    while True:
        current_time = datetime.now(timezone.utc)
        
        # Check for a new 15 minute candle
        #if current_time.minute % 15 == 0 and current_time.second < 10:
        if current_time.minute % 5 == 0 and current_time.second < 10:
            # Check if the 15-minute interval has changed since the last check
            if last_checked != current_time.minute:
                print(f"Current time: {current_time}")
                print("Checking for trade signals")
                price = get_candles(timeframe)
                price = calculate_indicators(price)
                ema_crossover(price)
                last_checked = current_time.minute  # Update to prevent re-triggering

        time.sleep(1)

In [9]:
run_bot()

Starting trading bot
Current time: 2025-11-05 09:05:00.185632+00:00
Checking for trade signals
Strategy conditions not met
Current time: 2025-11-05 09:10:00.209768+00:00
Checking for trade signals
Strategy conditions not met
Current time: 2025-11-05 09:15:00.769587+00:00
Checking for trade signals
Strategy conditions not met
Current time: 2025-11-05 09:20:00.263540+00:00
Checking for trade signals
Strategy conditions not met
Current time: 2025-11-05 09:25:00.832246+00:00
Checking for trade signals
Strategy conditions not met
Current time: 2025-11-05 09:30:00.703861+00:00
Checking for trade signals
Strategy conditions not met
Current time: 2025-11-05 09:35:00.278288+00:00
Checking for trade signals
Strategy conditions not met
Current time: 2025-11-05 09:40:00.916769+00:00
Checking for trade signals
Strategy conditions not met
Current time: 2025-11-05 09:45:00.386907+00:00
Checking for trade signals
Buy Signal: EMA 5 crossed above EMA 8
Placed order for GBP_JPY with stop loss at 200.193 

KeyboardInterrupt: 